# Sportify Lyrics Search

> Author: Rahul Ghosh

## Imports & Path Setup

In [ ]:
import sys
import torch

# Ensure the notebook can find your .py files in the src folder
sys.path.append("../src")

from model import LyricEmbeddingModel
from search import LyricSearchEngine

## Load Dataset & Build Model

In [ ]:
print(f"Using Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
MODEL = LyricEmbeddingModel()

MODEL.load_dataset("../data/spotify_lyrics.csv")

print("Generating embeddings... this may take a few minutes for 50k songs.")
MODEL.generate_embeddings()

df, embeddings, bert_model = MODEL.get_data()

## Initialize Search Engine

In [ ]:
search_engine = LyricSearchEngine(df)

## Test Prediction

In [ ]:
def evaluate_search(query, expected_song, top_k=5):
    print(f"\n--- Searching for: '{query}' ---")
    results = search_engine.search(query, top_k=top_k)
    
    # If results is a string (like "No matches found"), return False
    if isinstance(results, str):
        print(results)
        return False
        
    print(results)
    
    # CORRECTED LINE: Use .str before every string operation
    is_correct = results["track_name"].str.lower().str.strip().isin([expected_song.lower().strip()]).any()
    
    print(f"Result: {'✅ MATCH FOUND' if is_correct else '❌ MATCH NOT FOUND'}")
    return is_correct


In [ ]:
snippet_1 = "Half past twelve And I'm watching the late show in my flat all alone"
expected_1 = "Gimme! Gimme! Gimme!" # Replace with exact name in your CSV
evaluate_search(snippet_1, expected_1)

# Test 2: Partial/Fuzzy snippet (Testing the "Semantic" power of the model)
snippet_2 = "Like we used to do We don't love anymore"
expected_2 = "We don't talk anymore"
evaluate_search(snippet_2, expected_2)

# Test 3: Common lyrics
snippet_3 = "Wise men say only fools rush in"
expected_3 = "Can't Help Falling in Love"
evaluate_search(snippet_3, expected_3)


## Performance Summary

In [ ]:

test_cases = [
    {"snippet": "Half past twelve and I'm watching the late show", "expected": "Gimme! Gimme! Gimme!"},
    {"snippet": "Wise men say only fools rush in", "expected": "Can't Help Falling In Love"},
    {"snippet": "But we don't talk anymore like we used to do", "expected": "We Don't Talk Anymore"},
    {"snippet": "Just a small town girl living in a lonely world", "expected": "Don't Stop Believin'"},
    {"snippet": "Hello from the other side I must have called", "expected": "Hello"}
]

print("--- Running Performance Evaluation ---")
correct_count = 0
total_tests = len(test_cases)

for test in test_cases:
    # We use top_k=10 to be a bit more lenient for the summary
    success = evaluate_search(test["snippet"], test["expected"], top_k=5)
    if success:
        correct_count += 1

accuracy = (correct_count / total_tests) * 100

print("\n" + "="*30)
print(f"FINAL PERFORMANCE SUMMARY")
print(f"Total Queries Tested: {total_tests}")
print(f"Successful Matches:  {correct_count}")
print(f"Model Accuracy:      {accuracy:.2f}%")
print("="*30)


## Accuracy

In [ ]:
import matplotlib.pyplot as plt
import random

def evaluate_model(search_engine, df, n_samples=50, snippet_length=15, seed=42):
    # Reproducibility
    random.seed(seed)

    samples = df.sample(n=min(n_samples, len(df)), random_state=seed)

    top1, top3, top5 = 0, 0, 0

    for _, row in samples.iterrows():
        lyrics = row["lyrics"]
        true_track = row["track_name"]

        words = lyrics.split()

        # Random contiguous lyric snippet
        if len(words) > snippet_length:
            start_idx = random.randint(0, len(words) - snippet_length)
            snippet = " ".join(words[start_idx:start_idx + snippet_length])
        else:
            snippet = lyrics

        results = search_engine.search(snippet, top_k=5)
        retrieved_tracks = results["track_name"].tolist()

        if retrieved_tracks:
            if true_track == retrieved_tracks[0]:
                top1 += 1
            if true_track in retrieved_tracks[:3]:
                top3 += 1
            if true_track in retrieved_tracks[:5]:
                top5 += 1

    total = len(samples)

    report = {
        "Top-1 Accuracy": round(top1 / total * 100, 2),
        "Top-3 Accuracy": round(top3 / total * 100, 2),
        "Top-5 Accuracy": round(top5 / total * 100, 2),
        "Samples Evaluated": total
    }

    # ---------- BAR CHART ----------
    metrics = ["Top-1 Accuracy", "Top-3 Accuracy", "Top-5 Accuracy"]
    values = [report[m] for m in metrics]

    plt.figure(figsize=(7, 5))
    plt.bar(metrics, values)
    plt.ylim(0, 100)
    plt.xlabel("Evaluation Metric")
    plt.ylabel("Accuracy (%)")
    plt.title("Spotify Lyric Search – Retrieval Accuracy")

    for i, v in enumerate(values):
        plt.text(i, v + 1, f"{v}%", ha="center")

    plt.show()
    # --------------------------------

    return report


In [ ]:
report = evaluate_model(search_engine, df, n_samples=100)
report